<h1>Recommender system challenge PoliMi 2018</h1>

In [1]:
#install dependencies
!pip install kaggle

    100% |████████████████████████████████| 61kB 2.2MB/s 
    100% |████████████████████████████████| 235kB 5.2MB/s 
  Running setup.py bdist_wheel for kaggle ... - \ done
  Stored in directory: /root/.cache/pip/wheels/44/2c/df/22a6eeb780c36c28190faef6252b739fdc47145fd87a6642d4
  Running setup.py bdist_wheel for python-slugify ... - done
  Stored in directory: /root/.cache/pip/wheels/e3/65/da/2045deea3098ed7471eca0e2460cfbd3fdfe8c1d6fa6fcac92
Successfully built kaggle python-slugify


In [0]:
#import dependencies
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [4]:
#upload kaggle credentials
from google.colab import files
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"philcollins","key":"71407961b750f965467973ffcd24a705"}'}

In [0]:
#file configuration
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle

!chmod 600 ~/.kaggle/kaggle.json

In [6]:
#download dataset directly from kaggle
!kaggle competitions download -c recommender-system-2018-challenge-polimi

  0% 0.00/56.4k [00:00<?, ?B/s]
100% 56.4k/56.4k [00:00<00:00, 26.3MB/s]
  0% 0.00/392k [00:00<?, ?B/s]
100% 392k/392k [00:00<00:00, 47.5MB/s]
  0% 0.00/4.20M [00:00<?, ?B/s]
100% 4.20M/4.20M [00:00<00:00, 205MB/s]


In [8]:
#unzip
!ls
!unzip train.csv.zip

kaggle.json  sample_data  target_playlists.csv	tracks.csv  train.csv.zip
Archive:  train.csv.zip
  inflating: train.csv               


In [11]:
#see if we upload it correctly
train = pd.read_csv("train.csv")
train.head()

,playlist_id,track_id
0,0,14301
1,0,8360
2,0,12844
3,0,18397
4,0,1220


In [12]:
tracks = pd.read_csv("tracks.csv")
tracks.head()

,track_id,album_id,artist_id,duration_sec
0,0,6306,449,167
1,1,12085,4903,185
2,2,1885,6358,201
3,3,3989,1150,263
4,4,11633,4447,96
